In [2]:
!pip install pettingzoo[classic]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.1/251.1 kB 10.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.1/149.1 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 44.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 958.1/958.1 kB 36.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 847.8/847.8 kB 41.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 67.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 4.8 MB/s eta 0:00:00
  Created wheel for rlcard: filename=rlcard-1.0.5-py3-none-any.whl size=307096 sha256=62589bd15eae8580e3f5d4fda5fbe4f623efc8b0873096f1bbfe6f4b5ac748e0
  Stored in directory: /root/.cache/pip/wheels/22/1b/a9/cb9a2009aefc3e311979e3764bc8e095cc6a2541fdc3ce1f60
Successfully built rlcard
  Attempting uninstall: pygame
    Found existing installation: pygame 2.6.1
    Uninstalling pygame-2.6.1:
      S

In [12]:
from tensorflow import keras

In [ ]:
from keras.layers import Dense, Dropout

# Creación del modelo
model = keras.models.Sequential([
    Dense(64, activation = 'relu', input_shape = (72,)), # Capa Input de 72 neuronas (estado del juego)
    Dense(64, activation = 'relu'),
    Dropout(0.3),
    Dense(4, activation = 'softmax') # Capa Output de 4 neuronas (Acción a tomar) - Utilizamos softmax para que la suma de todas las probabilidades sea 1
])

In [ ]:
# Función para sacar la mejor acción a realizar (Para jugar el turno)
#   Argumentos: Estado del juego, acciones legales

# Implementación
#   Se le pasa a la capa Input del modelo el estado
#   El modelo devuelve la predicción probabilísitca de cada acción que cree que hay que tomar
#   Eliminar acciones ilegales (Multiplicar cada acción por su valor en la máscara)
#   Normalizar el resto de acciones para que sumen 1
#   Seleccionar una acción en base a una temperatura o algo así

In [3]:
from pettingzoo.classic import texas_holdem_v4
import numpy as np

In [11]:
env = texas_holdem_v4.env(num_players=3)
env.reset(seed=42)

for agent in env.agent_iter():
    observation, reward, termination, truncation, info = env.last()

    print(observation)

    print(len(observation['observation']))

    if termination or truncation:
        action = None
    else:
        mask = observation["action_mask"]
        # this is where you would insert your policy
        action = env.action_space(agent).sample(mask)

    env.step(action)
env.close()

{'observation': array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
       0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 1.,
       0., 0., 0., 0.], dtype=float32), 'action_mask': array([1, 1, 1, 0], dtype=int8)}
72
{'observation': array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 1.,
       0., 0., 0., 0.], dtype=float32), 'action_mask': array([1, 1, 1, 0], dtype=int8)}
72
{'observation': array([0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0

In [ ]:
# Definir la acción basada en el estado actual
def elegir_accion(estado, acciones_disponibles, q_table, epsilon):
    # Explorar o explotar según el valor de epsilon
    if np.random.rand() < epsilon:  # Exploración
        return np.random.choice(acciones_disponibles)
    else:  # Explotación
        # Si el estado no está en q_table, inicializar
        if estado not in q_table:
            q_table[estado] = {accion: 0 for accion in acciones_disponibles}
        q_values = q_table[estado]
        return acciones_disponibles[np.argmax(q_values)]  # Elegir la acción con mejor Q-valor

In [ ]:
# Crear el entorno de Póker Texas Hold'em
env = texas_holdem_v4.env(num_players=3)
env.reset()

# Inicializar la tabla Q
q_table = {}

# Parámetros de aprendizaje
alpha = 0.1  # Tasa de aprendizaje
gamma = 0.9  # Factor de descuento
epsilon = 0.7  # Probabilidad de exploración
num_episodios = 100  # Número de episodios

# Entrenar el modelo
for episodio in range(num_episodios):
    env.reset()
    recompensa_total = 0
    for agente in env.agent_iter():
      resultado, recompensa, done, _, info = env.last()

      # Extraer el estado y la máscara de acciones
      estado = resultado['observation']  # Estado actual del entorno
      acciones = resultado['action_mask']  # Acciones legales para realizar
      acciones_disponibles = [i for i, legal in enumerate(acciones) if legal]

      if done:
        env.step(None)
        continue

      # Serializar estado
      estado_serializado = str(estado)

      # Inicializar en la Q-table si no existe
      if estado_serializado not in q_table:
          q_table[estado_serializado] = {accion: 0 for accion in acciones_disponibles}

      # Elegir acción
      accion = elegir_accion(estado_serializado, acciones_disponibles, q_table, epsilon)
      # Tomar acción
      env.step(accion)

      # Asegurar que la acción está en el diccionario (prevención de errores)
      if accion not in q_table[estado_serializado]:
          q_table[estado_serializado][accion] = 0

      # Obtener el nuevo estado
      resultado, _, done, _, _ = env.last()
      nuevo_estado = resultado['observation']
      nuevo_estado_serializado = str(nuevo_estado)

      nuevas_acciones = resultado['action_mask']
      nuevas_acciones_dsiponibles = [i for i, legal in enumerate(nuevas_acciones) if legal]

      if nuevo_estado_serializado not in q_table:
          q_table[nuevo_estado_serializado] = {accion: 0 for accion in nuevas_acciones_dsiponibles}

      # Actualizar la Q-table
      if not q_table[nuevo_estado_serializado]:
        max_q_siguiente = 0
      else:
        max_q_siguiente = max(q_table[nuevo_estado_serializado].values()) if not done else 0
      valor = alpha * (recompensa + gamma * max_q_siguiente - q_table[estado_serializado][accion])
      print(f"Recompensa: {recompensa}, Max Q siguiente: {max_q_siguiente}, Q actual: {q_table[estado_serializado][accion]}, Valor calculado: {valor}")
      q_table[estado_serializado][accion] += valor

      recompensa_total += recompensa

    print(f'Episodio {episodio+1}, recompensa total: {recompensa_total}')
    #break

Recompensa: 0, Max Q siguiente: 0, Q actual: 0, Valor calculado: 0.0
Recompensa: 0, Max Q siguiente: 0, Q actual: 0, Valor calculado: 0.0
Recompensa: 0, Max Q siguiente: 0, Q actual: 0, Valor calculado: 0.0
Recompensa: 0, Max Q siguiente: 0, Q actual: 0, Valor calculado: 0.0
Recompensa: 0, Max Q siguiente: 0, Q actual: 0, Valor calculado: 0.0
Recompensa: 0, Max Q siguiente: 0, Q actual: 0, Valor calculado: 0.0
Recompensa: 0, Max Q siguiente: 0, Q actual: 0, Valor calculado: 0.0
Recompensa: 0, Max Q siguiente: 0, Q actual: 0, Valor calculado: 0.0
Recompensa: 0, Max Q siguiente: 0, Q actual: 0, Valor calculado: 0.0
Episodio 1, recompensa total: 0
Recompensa: 0, Max Q siguiente: 0, Q actual: 0, Valor calculado: 0.0
Recompensa: 0, Max Q siguiente: 0, Q actual: 0, Valor calculado: 0.0
Recompensa: 0, Max Q siguiente: 0, Q actual: 0, Valor calculado: 0.0
Recompensa: 0, Max Q siguiente: 0, Q actual: 0, Valor calculado: 0.0
Episodio 2, recompensa total: 0
Recompensa: 0, Max Q siguiente: 0, Q ac

In [ ]:
# Evaluar el modelo
estado = env.reset()
estado_tuple = tuple(estado[0]['obs'])
recompensa_total = 0
done = False
while not done:
    acciones_disponibles = list(range(env.action_num))
    accion = accion_estado(estado_tuple, q_table, 0, acciones_disponibles)  # Sin exploración (epsilon=0)
    estado, recompensa, done, _ = env.step(accion)
    estado_tuple = tuple(estado[0])
    recompensa_total += recompensa[0]

print(f'Recompensa final tras evaluación: {recompensa_total}')